In [ ]:
!pip install kaggle

Importing the dependencies

In [ ]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Data Collection

In [ ]:
kaggle_dictionary = json.load(open("kaggle.json"))

In [ ]:
kaggle_dictionary.values()

dict_values(['gauravkumar2021', 'b1b1462ba689316de6b0a9b83c582fd1'])

In [ ]:
# setup kaggle credentials as environment variables
os.environ['KAGGLE_USERNAME'] = kaggle_dictionary['username']
os.environ['KAGGLE_KEY'] = kaggle_dictionary['key']

In [ ]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
imdb-dataset-of-50k-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!ls

 drive		     imdb-dataset-of-50k-movie-reviews.zip   sample_data
'IMDB Dataset.csv'   kaggle.json


In [ ]:
# unzip the dataset file
with ZipFile("imdb-dataset-of-50k-movie-reviews.zip", "r") as zip_ref:
  zip_ref.extractall()

In [ ]:
!ls

 drive		     imdb-dataset-of-50k-movie-reviews.zip   sample_data
'IMDB Dataset.csv'   kaggle.json


In [ ]:
data = pd.read_csv("IMDB Dataset.csv")

In [ ]:
data.shape

(50000, 2)

In [ ]:
data.head()

review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive

In [ ]:
data["sentiment"].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [ ]:
data.replace({"positive": 1, "negative": 0}, inplace=True)

/tmp/ipython-input-1021368684.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"positive": 1, "negative": 0}, inplace=True)


In [ ]:
data.head()

review  sentiment
0  One of the other reviewers has mentioned that ...          1
1  A wonderful little production. <br /><br />The...          1
2  I thought this was a wonderful way to spend ti...          1
3  Basically there's a family where a little boy ...          0
4  Petter Mattei's "Love in the Time of Money" is...          1

In [ ]:
#split data into train data and test data
train_data, test_data = train_test_split(data, test_size = 0.2, random_state = 42)

In [ ]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


Data Preprocessing

In [ ]:
# Tokenize the text data
tokenizer = Tokenizer(num_words = 5000) ##convert text into number,most common 5000 words
tokenizer.fit_on_texts(train_data["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen = 200)
x_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen = 200)

In [ ]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [ ]:
print(x_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [ ]:
y_train = train_data["sentiment"]
y_test = test_data["sentiment"]

In [ ]:
print(y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


In [ ]:
print(y_test)

33553    1
9427     1
199      0
12447    1
39489    0
        ..
28567    0
25079    1
18707    1
15200    0
5857     1
Name: sentiment, Length: 10000, dtype: int64


**LSTM - Long Short-Term Memory**(RNN)

In [ ]:
## Build the model

model = Sequential()
model.add(Embedding(input_dim = 5000, output_dim = 128,input_length = 200))
model.add(LSTM(128, dropout = 0.2, recurrent_dropout =0.2))
model.add(Dense(1, activation ="sigmoid"))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_shape=(200,)))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:100: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ (None, 200, 128)       │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 771,713 (2.94 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model = Sequential([
    Embedding(input_dim=5000, output_dim=128, input_shape=(200,)),
    LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    Dense(1, activation="sigmoid")
])
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ (None, 200, 128)       │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 771,713 (2.94 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# compile the model
model.compile(optimizer = "adam", loss ="binary_crossentropy", metrics = ["accuracy"])

**Training the model**

In [ ]:
model.fit(X_train, y_train, epochs = 5, batch_size = 64, validation_split = 0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 337s 666ms/step - accuracy: 0.7319 - loss: 0.5195 - val_accuracy: 0.7994 - val_loss: 0.4330
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 383s 667ms/step - accuracy: 0.8445 - loss: 0.3679 - val_accuracy: 0.8329 - val_loss: 0.3839
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 382s 668ms/step - accuracy: 0.8757 - loss: 0.3042 - val_accuracy: 0.8583 - val_loss: 0.3323
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 384s 672ms/step - accuracy: 0.9037 - loss: 0.2492 - val_accuracy: 0.8673 - val_loss: 0.3212
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 376s 661ms/step - accuracy: 0.9173 - loss: 0.2186 - val_accuracy: 0.8746 - val_loss: 0.3109


**Model Evaluation**

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test Loss : {loss}")
print(f"Test accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 28s 89ms/step - accuracy: 0.8762 - loss: 0.2993
Test Loss : 0.29758673906326294
Test accuracy: 0.8763999938964844


Building a predictive System

In [ ]:
def predict_sentiment(review):
  #tokenize and pad the review
  sequence = tokenizer.texts_to_sequences([review])
  padding_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padding_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [ ]:
# example usage
new_review = "This movie was fantastic. i loved it"
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 447ms/step
The sentiment of the review is: positive


In [ ]:
# example usage
new_review = "This movie is not that good."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
The sentiment of the review is: negative
